In [ ]:
#hide
from pySplitting.core import *
from sympy.matrices.dense import symarray

# pySplitting

> A project to have python code that generates the order conditions for exponential splitting methods with the techniques of M. Thalhammer


This file will become your README and also the index of your documentation.

## Install

`pip pySplitting`

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
alpha([1,1,1])

1/6

In [ ]:
avec=[*symarray('a',3)]
#bvec=[*symarray('b',3)]
#print(CreateConditions(3,avec,bvec))
#print("=========================")
#print(CreateConditions(3,avec,bvec,CreateMuVectors))